In [388]:
import json 
import csv 
import pandas
import pandas as pd

In [389]:
data = pd.read_csv('Desktop/Sarcasm_Corr/Twitter_Training2Contexts.csv')
pd.options.display.max_columns = None
df = data[['context/0','response','label']].copy()
df.head()

,context/0,response,label
0,@USER If your child isn't named Barron ... #Be...,@USER @USER @USER I don't get this .. obviousl...,SARCASM
1,@USER @USER having to make up excuses of why y...,@USER @USER trying to protest about . Talking ...,SARCASM
2,@USER I ’ ll remember to not support you at th...,@USER @USER @USER He makes an insane about of ...,SARCASM
3,@USER But not half as stupid as Schiff looks ....,@USER @USER Meanwhile Trump won't even release...,SARCASM
4,@USER They already did . Obama said many times...,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,SARCASM


In [390]:
"""### Vader """
def vader(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    vader = SentimentIntensityAnalyzer()
    score = vader.polarity_scores(sentence)
    return score

# creating lists to keep pos, neu, neg, and compound scores --- later to be used to create a dataframe


# extracting vader scores for each entry in the data
# (we're not using context yet.)
# note that the compound score is rescaled to the [0,1] range
# some classiifers don't take negative values (e.g., MultinomialNB)


def vader_scores(df):
    vs_compound = []
    vs_pos = []
    vs_neu = []
    vs_neg = []
    for row in df:
        score = vader(row)
        neg = float(score['neg'])
        vs_neg.append(neg)
        neu =float(score['neu'])
        vs_neu.append(neu)
        pos =float(score['pos'])
        vs_pos.append(pos)
        compound = float((score['compound']+1)/2) # rescaling to the [0,1] range
        vs_compound.append(compound)
    return vs_compound, vs_pos, vs_neu, vs_neg


In [391]:
cvs_compound, cvs_pos, cvs_neu, cvs_neg = vader_scores(df['context/0'])  #compute vader scores for context/0

df = df.assign(cntx_vader_neu = cvs_neu, cntx_vader_pos = cvs_pos, cntx_vader_neg = cvs_neg, cntx_vader_compound = cvs_compound)

In [392]:
rvs_compound, rvs_pos, rvs_neu, rvs_neg = vader_scores(df['response'])  #compute vader scores for response
df = df.assign(resp_vader_neu = rvs_neu, resp_vader_pos = rvs_pos, resp_vader_neg = rvs_neg, resp_vader_compound = rvs_compound)


In [393]:
df1 = df[df["label"]=="SARCASM"].copy()  #Divide DATA by label 
df2 = df[df["label"]=="NOT_SARCASM"].copy() #Divide DATA by label

# Compare Response to Context

## VADER

In [394]:
from scipy import stats

def correlation_calc(response, context, a):   #Pearson
    r_list = response.tolist()
    c_list = context.tolist()
    correlation,p_value = stats.pearsonr(r_list,c_list)
    return a, correlation

def correlation_calcSpearman(response, context, a):   #Spearman
    r_list = response.tolist()
    c_list = context.tolist()
    correlation,p_value = stats.spearmanr(r_list,c_list)
    return a, correlation


Compute the Correlation of the Vader Compound Scores for Sarcastic Responses and the Sarcastic Context that preceeded them. Compute the correlation of the Non-Sarcastic Responses to their corresponding contexts. 
<br>
Continue doing this for all Vader Values (Negative, Positive, Neutral, and Compound)

In [395]:
print('Pearson')
print('Compound')
print(correlation_calc(df1['resp_vader_compound'], df1['cntx_vader_compound'], "Sarcasm Vader Compound Correlation: "))
print(correlation_calc(df2['resp_vader_compound'], df2['cntx_vader_compound'], "Not Sarcasm Vader Compound Correlation: "))
print()
print('Positive')
print(correlation_calc(df1['resp_vader_pos'], df1['cntx_vader_pos'], "Sarcasm Vader Positive Correlation: "))
print(correlation_calc(df2['resp_vader_pos'], df2['cntx_vader_pos'], "Not Sarcasm Vader Positive Correlation: "))
print()
print('Negative')
print(correlation_calc(df1['resp_vader_neg'], df1['cntx_vader_neg'], "Sarcasm Vader Negative Correlation: "))
print(correlation_calc(df2['resp_vader_neg'], df2['cntx_vader_neg'], "Not Sarcasm Vader Negative Correlation: "))
print()
print('Neutral')
print(correlation_calc(df1['resp_vader_neu'], df1['cntx_vader_neu'], "Sarcasm Vader Neutral Correlation: "))
print(correlation_calc(df2['resp_vader_neu'], df2['cntx_vader_neu'], "Not Sarcasm Vader Neutral Correlation: "))


print()
print()

print('Spearman')
print('Compound')
print(correlation_calcSpearman(df1['resp_vader_compound'], df1['cntx_vader_compound'], "Sarcasm Vader Compound Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_compound'], df2['cntx_vader_compound'], "Not Sarcasm Vader Compound Correlation: "))
print()
print('Positive')
print(correlation_calcSpearman(df1['resp_vader_pos'], df1['cntx_vader_pos'], "Sarcasm Vader Positive Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_pos'], df2['cntx_vader_pos'], "Not Sarcasm Vader Positive Correlation: "))
print()
print('Negative')
print(correlation_calcSpearman(df1['resp_vader_neg'], df1['cntx_vader_neg'], "Sarcasm Vader Negative Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_neg'], df2['cntx_vader_neg'], "Not Sarcasm Vader Negative Correlation: "))
print()
print('Neutral')
print(correlation_calcSpearman(df1['resp_vader_neu'], df1['cntx_vader_neu'], "Sarcasm Vader Neutral Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_neu'], df2['cntx_vader_neu'], "Not Sarcasm Vader Neutral Correlation: "))


Pearson
Compound
('Sarcasm Vader Compound Correlation: ', 0.07605447332942038)
('Not Sarcasm Vader Compound Correlation: ', 0.2749945641751499)

Positive
('Sarcasm Vader Positive Correlation: ', 0.0872712783097193)
('Not Sarcasm Vader Positive Correlation: ', 0.28833031334280174)

Negative
('Sarcasm Vader Negative Correlation: ', 0.13894980844486815)
('Not Sarcasm Vader Negative Correlation: ', 0.222379548228165)

Neutral
('Sarcasm Vader Neutral Correlation: ', 0.15577725670092682)
('Not Sarcasm Vader Neutral Correlation: ', 0.2080424030509918)


Spearman
Compound
('Sarcasm Vader Compound Correlation: ', 0.07210957829895888)
('Not Sarcasm Vader Compound Correlation: ', 0.30928573118032054)

Positive
('Sarcasm Vader Positive Correlation: ', 0.0689342310373311)
('Not Sarcasm Vader Positive Correlation: ', 0.25313823098823024)

Negative
('Sarcasm Vader Negative Correlation: ', 0.11009089507947474)
('Not Sarcasm Vader Negative Correlation: ', 0.23755676598443232)

Neutral
('Sarcasm Vader N

# VAD

Valence/Arousal/Dominance

Using Warriner et al.'s dictionary for norms of valence, arousal, and dominance (VAD) for 13,915 English lemmas, we compute the average VAD scores for each tweet. 

Next, we compute the correlation between the valence/arousal/dominance score of the sarcastic responses and their corresponding preceeding contexts, and compare this to the correlation between the correlation of the VAD score of the non-sarcastic responses & contexts. We hypothesize that the Not-Sarcastic responses will have a higher correlation with their preceeding context since Sarcasm can be marked by a change in emotion.


In [397]:

# Vad will pull all the words that are found in the dictionary, then sum up all the scores, and divide by # of words that matched dictionary entry
import pandas as pd
import nltk



"""### Dataframe
Columns: Context & Response
"""

df.loc[:,'tokenized_context']  = df.loc[:,'context/0'].str.replace('[^\w\s]','')  #get rid of punctuation
df['tokenized_response']  = df.loc[:,'response'].str.replace('[^\w\s]','')  #get rid of punctuation

"""tokenize context & response"""
df.loc[:,'tokenized_context'] = df.apply(lambda row: nltk.word_tokenize(row['tokenized_context']), axis=1) #tokenize
df.loc[:,'tokenized_response'] = df.apply(lambda row: nltk.word_tokenize(row['tokenized_response']), axis=1) #tokenize



from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag))) for (word,pos_tag) in pos] for pos in pos_tokens]
        #pos_tokens = [ [(lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]

        return pos_tokens

lemmatizer = WordNetLemmatizer()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

#step 2 lemmatization

df.loc[:,'lemma_response'] = lemmatization_using_pos_tagger.pos_tag((df.loc[:,'tokenized_response'])) #response lemma
df.loc[:,'lemma_context'] = lemmatization_using_pos_tagger.pos_tag((df.loc[:,'tokenized_context'])) #context lemma


"""VAD dictionary"""
import csv
reader = csv.reader(open('Desktop/UPDATED_NLP_COURSE/vad1.csv'))

d = {}
for row in reader:
    key = row[1]
    value = row[2:]
    d[key] = value
    
d.pop('Word')          


""" CHECK LEMMAS AGAINST VAD DICTIONARY"""

"""response"""

list_totals = [[(list(map(float,d[token]))) for token in row if token in d] for row in df['lemma_response'].array]

def find_sum(x):
    total = [[num[x] for num in l]for l in list_totals]
    full_list = []
    for i in total:
        if len(i) != 0:
            full_list.append(i)
        else:
            i = [5.0,5.0,5.0]    #if there is a tweet that has nothing, assign average value
            full_list.append(i)
    average = [((sum(x))/(len(x))) for x in full_list]     #sum of all values divided by how many there were
    return average


"""Response"""
vr_total = find_sum(0)
ar_total = find_sum(1)
dr_total = find_sum(2)

df.loc[:,'valence_response'] = vr_total
df.loc[:,'arousal_response'] = ar_total
df.loc[:,'dominance_response'] = dr_total



"""Context"""

list_totals = [[(list(map(float,d[token]))) for token in row if token in d] for row in df['lemma_context'].array]


vc_total = find_sum(0)
ac_total = find_sum(1)
dc_total = find_sum(2)


df.loc[:,'valence_context'] = vc_total
df.loc[:,'arousal_context'] = ac_total
df.loc[:,'dominance_context'] = dc_total

In [398]:
df.head()

,context/0,response,label,cntx_vader_neu,cntx_vader_pos,cntx_vader_neg,cntx_vader_compound,resp_vader_neu,resp_vader_pos,resp_vader_neg,resp_vader_compound,tokenized_context,tokenized_response,lemma_response,lemma_context,valence_response,arousal_response,dominance_response,valence_context,arousal_context,dominance_context
0,@USER If your child isn't named Barron ... #Be...,@USER @USER @USER I don't get this .. obviousl...,SARCASM,0.832,0.000,0.168,0.30625,0.796,0.204,0.000,0.87530,"[USER, If, your, child, isnt, named, Barron, B...","[USER, USER, USER, I, dont, get, this, obvious...","[USER, USER, USER, I, dont, get, this, obvious...","[USER, If, your, child, isnt, name, Barron, Be...",6.222500,3.695000,5.965000,6.820000,3.680000,5.633333
1,@USER @USER having to make up excuses of why y...,@USER @USER trying to protest about . Talking ...,SARCASM,1.000,0.000,0.000,0.50000,0.744,0.000,0.256,0.11985,"[USER, USER, having, to, make, up, excuses, of...","[USER, USER, trying, to, protest, about, Talki...","[USER, USER, try, to, protest, about, Talking,...","[USER, USER, have, to, make, up, excuse, of, w...",5.190000,3.713333,5.733333,5.481667,3.801667,5.316667
2,@USER I ’ ll remember to not support you at th...,@USER @USER @USER He makes an insane about of ...,SARCASM,0.816,0.000,0.184,0.34555,0.824,0.000,0.176,0.27130,"[USER, I, ll, remember, to, not, support, you,...","[USER, USER, USER, He, makes, an, insane, abou...","[USER, USER, USER, He, make, an, insane, about...","[USER, I, ll, remember, to, not, support, you,...",5.246667,5.223333,5.426667,5.815000,2.977500,5.562500
3,@USER But not half as stupid as Schiff looks ....,@USER @USER Meanwhile Trump won't even release...,SARCASM,0.708,0.253,0.039,0.92570,0.870,0.000,0.130,0.24470,"[USER, But, not, half, as, stupid, as, Schiff,...","[USER, USER, Meanwhile, Trump, wont, even, rel...","[USER, USER, Meanwhile, Trump, wont, even, rel...","[USER, But, not, half, as, stupid, a, Schiff, ...",5.661111,4.428889,5.907778,6.292000,4.144667,5.931333
4,@USER They already did . Obama said many times...,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,SARCASM,0.841,0.058,0.100,0.39885,0.807,0.193,0.000,0.83525,"[USER, They, already, did, Obama, said, many, ...","[USER, USER, Pretty, Sure, the, AntiLincoln, C...","[USER, USER, Pretty, Sure, the, AntiLincoln, C...","[USER, They, already, do, Obama, say, many, ti...",5.000000,5.000000,5.000000,5.740625,4.018125,5.687500


In [106]:
# # Divide data by label again 
df3 = df[df["label"]=="SARCASM"].copy()  #Divide DATA by label
df4 = df[df["label"]=="NOT_SARCASM"].copy() #Divide DATA by label

In [107]:
print("Valence")
print(correlation_calc(df3['valence_response'], df3['valence_context'], "Sarcasm Valence Correlation"))
print(correlation_calc(df4['valence_response'], df4['valence_context'], "Not Sarcasm Valence Correlation"))
print()
print("Dominance")
print(correlation_calc(df3['dominance_response'], df3['dominance_context'], "Sarcasm Dominance Correlation"))
print(correlation_calc(df4['dominance_response'], df4['dominance_context'], "Not Sarcasm Dominance Correlation"))
print()
print("Arousal")
print(correlation_calc(df3['arousal_response'], df3['arousal_context'], "Sarcasm Arousal Correlation"))
print(correlation_calc(df4['arousal_response'], df4['arousal_context'], "Not Sarcasm Arousal Correlation"))

Valence
('Sarcasm Valence Correlation', 0.14586568593344873)
('Not Sarcasm Valence Correlation', 0.3009481608205475)

Dominance
('Sarcasm Dominance Correlation', 0.1341486111329201)
('Not Sarcasm Dominance Correlation', 0.20535444999434416)

Arousal
('Sarcasm Arousal Correlation', 0.12180082280460452)
('Not Sarcasm Arousal Correlation', 0.13527132448249019)


# LIWC

In [331]:
import csv 
from liwc import Liwc
df5 = data[['context/0','response','label']].copy()

df5.loc[:,'tokenized_context']  = df5.loc[:,'context/0'].str.replace('[^\w\s]','')  #get rid of punctuation
df5.loc[:,'tokenized_response']  = df5.loc[:,'response'].str.replace('[^\w\s]','')  #get rid of punctuation


"""tokenize context & response"""
df5.loc[:,'tokenized_context'] = df5.apply(lambda row: nltk.word_tokenize(row['tokenized_context']), axis=1) #tokenize
df5.loc[:,'tokenized_response'] = df5.apply(lambda row: nltk.word_tokenize(row['tokenized_response']), axis=1) #tokenize


lwc = Liwc("Desktop/UPDATED_NLP_COURSE/liwc_dictionaries_shared/LIWC2007_English100131.dic") #liwc dictionary


def liwc_parse(data):    #count which categories appear in each tweet & how many times
    liwcresults =[]
    for token in data:  #response scores
        results = (lwc.parse(token))
        liwcresults.append(results)
    return liwcresults

liwc_results = liwc_parse(df5.loc[:,'tokenized_response'])

def liwc_unique_keys(data):   #collect all field names
    liwc_keys = []
    for key_val in range(len(data)):
        for key in data[key_val].keys():
            if key not in liwc_keys:
                liwc_keys.append(key)
    return liwc_keys

liwc_keys = liwc_unique_keys(liwc_results)


#look at each category in range 3,000, if the value is not in the list of keys, append new key. 

with open('liwc_feature_chart_response.csv', 'w') as csv_file:  
    dict_writer = csv.DictWriter(csv_file, liwc_keys)     #liwc keys will be column heasders   
    writer = csv.writer(csv_file)
    dict_writer.writeheader()
    dict_writer.writerows(liwc_results)      #liwc results will be the rows
    
liwcdata = pd.read_csv("liwc_feature_chart_response.csv")   #dataframe of liwc feeatures
df5 = pd.concat([liwcdata, df5], axis=1, sort=False)   #combine dataframes
df5.fillna(0, inplace=True)  #put zeroes wherever there are no values


In [332]:
keep_same = {'context/0', 'response', 'label', 'tokenized_context', 'tokenized_response'}

In [333]:
df5.columns = ['{}{}'.format(c, '' if c in keep_same else '_resp') for c in df5.columns]

In [334]:
df6 = data[['context/0','response','label']].copy()

liwc_results = liwc_parse(df5.loc[:,'tokenized_context'])
liwc_keys = liwc_unique_keys(liwc_results)

with open('liwc_feature_chart_context.csv', 'w') as csv_file:  
    dict_writer = csv.DictWriter(csv_file, liwc_keys)     #liwc keys will be column heasders   
    writer = csv.writer(csv_file)
    dict_writer.writeheader()
    dict_writer.writerows(liwc_results)      #liwc results will be the rows
    
liwcdata = pd.read_csv("liwc_feature_chart_context.csv")   #dataframe of liwc feeatures
df6 = pd.concat([liwcdata, df6], axis=1, sort=False)   #combine dataframes
df6.fillna(0, inplace=True)  #put zeroes wherever there are no values

df6.columns = ['{}{}'.format(c, '' if c in keep_same else '_cntx') for c in df6.columns]

In [335]:
df5 = df5.drop(['context/0', 'response','tokenized_context','tokenized_response'], axis=1)
df6 = df6.drop(['context/0', 'response','label'], axis=1)

In [338]:
final_df = pd.concat([df5, df6], axis=1)
col_name = 'label'
first_col = final_df.pop(col_name)
final_df.insert(0, col_name, first_col)

In [340]:
final_df_s = final_df[final_df["label"]=="SARCASM"].copy()  #Divide DATA by label
final_df_ns = final_df[final_df["label"]=="NOT_SARCASM"].copy() #Divide DATA by label



In [342]:
final_df_s

,label,verb_resp,funct_resp,auxverb_resp,present_resp,negate_resp,pronoun_resp,ipron_resp,cogmech_resp,certain_resp,ppron_resp,you_resp,social_resp,affect_resp,posemo_resp,conj_resp,tentat_resp,excl_resp,past_resp,future_resp,discrep_resp,motion_resp,relativ_resp,space_resp,preps_resp,incl_resp,adverb_resp,insight_resp,shehe_resp,achieve_resp,negemo_resp,anger_resp,they_resp,cause_resp,article_resp,money_resp,work_resp,percept_resp,hear_resp,swear_resp,time_resp,leisure_resp,death_resp,number_resp,see_resp,sad_resp,quant_resp,anx_resp,assent_resp,bio_resp,health_resp,filler_resp,body_resp,friend_resp,sexual_resp,we_resp,home_resp,feel_resp,inhib_resp,humans_resp,i_resp,nonfl_resp,family_resp,relig_resp,ingest_resp,funct_cntx,pronoun_cntx,ppron_cntx,you_cntx,social_cntx,humans_cntx,verb_cntx,auxverb_cntx,present_cntx,negate_cntx,cogmech_cntx,discrep_cntx,affect_cntx,posemo_cntx,quant_cntx,preps_cntx,cause_cntx,space_cntx,relativ_cntx,past_cntx,insight_cntx,excl_cntx,article_cntx,work_cntx,number_cntx,conj_cntx,negemo_cntx,anger_cntx,percept_cntx,see_cntx,ipron_cntx,achieve_cntx,they_cntx,incl_cntx,shehe_cntx,time_cntx,hear_cntx,adverb_cntx,certain_cntx,filler_cntx,inhib_cntx,leisure_cntx,motion_cntx,i_cntx,bio_cntx,sexual_cntx,tentat_cntx,future_cntx,ingest_cntx,we_cntx,assent_cntx,feel_cntx,relig_cntx,swear_cntx,money_cntx,body_cntx,home_cntx,death_cntx,health_cntx,sad_cntx,anx_cntx,family_cntx,friend_cntx,nonfl_cntx
0,SARCASM,7.0,13.0,3.0,5.0,1.0,5.0,1.0,6.0,1.0,4.0,3.0,4.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SARCASM,2.0,11.0,1.0,2.0,0.0,5.0,1.0,3.0,0.0,4.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,1.0,1.0,3.0,0.0,3.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SARCASM,1.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SARCASM,5.0,10.0,3.0,1.0,1.0,4.0,0.0,2.0,1.0,4.0,0.0,5.0,1.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,6.0,4.0,0.0,5.0,1.0,7.0,3.0,5.0,2.0,9.0,0.0,4.0,3.0,0.0,6.0,3.0,0.0,0.0,2.0,1.0,3.0,0.0,1.0,1.0,4.0,1.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SARCASM,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,2.0,1.0,4.0,0.0,5.0,3.0,1.0,0.0,3.0,0.0,1.0,1.0,3.0,5.0,0.0,2.0,5.0,4.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [357]:
sarcasm_response = final_df_s.iloc[:,1:65]
sarcasm_context = final_df_s.iloc[:,65:]

sarcasm_response = sarcasm_response.reindex(sorted(sarcasm_response.columns), axis=1)
sarcasm_context= sarcasm_context.reindex(sorted(sarcasm_context.columns), axis=1)

In [358]:
not_sarcasm_response = final_df_ns.iloc[:,1:65]
not_sarcasm_context = final_df_ns.iloc[:,65:]

not_sarcasm_response = not_sarcasm_response.reindex(sorted(not_sarcasm_response.columns), axis=1)
not_sarcasm_context= not_sarcasm_context.reindex(sorted(not_sarcasm_context.columns), axis=1)

In [376]:

def liwc_corr(response, context):
    Scores_List = []
    col_len = len(response.columns)
    for i in range(col_len):
        r_list = response.iloc[:,i].tolist()
        c_list = context.iloc[:,i].tolist()
        correlation, p_value = stats.pearsonr(r_list,c_list)
        Scores_List.append(((response.columns[i], context.columns[i],correlation)))
    return Scores_List

Sarcasm_Scores_List = liwc_corr(sarcasm_response, sarcasm_context)

Not_Sarcasm_Scores_List = liwc_corr(not_sarcasm_response, not_sarcasm_context)


In [377]:
Sarcasm_Scores_List

[('achieve_resp', 'achieve_cntx', 0.1737161929777084),
 ('adverb_resp', 'adverb_cntx', 0.12193475345940082),
 ('affect_resp', 'affect_cntx', 0.18445642515504806),
 ('anger_resp', 'anger_cntx', 0.20048015337057512),
 ('anx_resp', 'anx_cntx', 0.07301330718282943),
 ('article_resp', 'article_cntx', 0.1870866155515437),
 ('assent_resp', 'assent_cntx', 0.02416400857128782),
 ('auxverb_resp', 'auxverb_cntx', 0.09846880465342057),
 ('bio_resp', 'bio_cntx', 0.18238292344695495),
 ('body_resp', 'body_cntx', 0.08540080557555985),
 ('cause_resp', 'cause_cntx', 0.11096942273391275),
 ('certain_resp', 'certain_cntx', 0.0753806858515828),
 ('cogmech_resp', 'cogmech_cntx', 0.22890005402680802),
 ('conj_resp', 'conj_cntx', 0.13937904555204755),
 ('death_resp', 'death_cntx', 0.20149910200267696),
 ('discrep_resp', 'discrep_cntx', 0.09945624130638013),
 ('excl_resp', 'excl_cntx', 0.06870192739355996),
 ('family_resp', 'family_cntx', 0.16385418050746725),
 ('feel_resp', 'feel_cntx', 0.06432711210969419),

In [379]:
Not_Sarcasm_Scores_List

[('achieve_resp', 'achieve_cntx', 0.20968216410453894),
 ('adverb_resp', 'adverb_cntx', 0.10040129558147906),
 ('affect_resp', 'affect_cntx', 0.23684884098175382),
 ('anger_resp', 'anger_cntx', 0.1297424644866993),
 ('anx_resp', 'anx_cntx', 0.11607877489590426),
 ('article_resp', 'article_cntx', 0.1415219446833923),
 ('assent_resp', 'assent_cntx', 0.10754202554294308),
 ('auxverb_resp', 'auxverb_cntx', 0.22892615816889592),
 ('bio_resp', 'bio_cntx', 0.13715041485486307),
 ('body_resp', 'body_cntx', 0.06860147687667714),
 ('cause_resp', 'cause_cntx', 0.10042093195021502),
 ('certain_resp', 'certain_cntx', 0.06622719863343475),
 ('cogmech_resp', 'cogmech_cntx', 0.20350499209346565),
 ('conj_resp', 'conj_cntx', 0.1266598376837689),
 ('death_resp', 'death_cntx', 0.1420754584101624),
 ('discrep_resp', 'discrep_cntx', 0.10646218587196912),
 ('excl_resp', 'excl_cntx', 0.11991222724091602),
 ('family_resp', 'family_cntx', 0.20952132360837916),
 ('feel_resp', 'feel_cntx', 0.09594015160381922),


In [385]:

    
NS_Scores_List = [i[2] for i in Not_Sarcasm_Scores_List]
S_Scores_List = [i[2] for i in Sarcasm_Scores_List]



In [386]:
def Average(lst): 
    return sum(lst) / len(lst)

print("NotSarcasm Average", (Average(NS_Scores_List)))
print()
print("Sarcasm Average", (Average(S_Scores_List)))



NotSarcasm Average 0.1676196197536272

Sarcasm Average 0.13662030140000075
